<a href="https://colab.research.google.com/github/fabriciosantana/mcdia/blob/main/05-iag/nemotron_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NVIDIA Nemotron - Exemplos de Uso

O **Nemotron** é uma família de modelos da NVIDIA com arquitetura híbrida Mamba-Transformer MoE.

Este notebook mostra como usar o Nemotron via NVIDIA NIM API.

# 1. Importar bibliotecas

In [5]:
import os
from google.colab import userdata

import requests

## 1. Configurar API Key


In [6]:
def get_nvidia_api_key():
    key = None
    if key:
        key = os.environ.get("NVIDIA_API_KEY")
    else:
        key = userdata.get('NVIDIA_API_KEY')

    return key

NVIDIA_API_KEY = get_nvidia_api_key()

if NVIDIA_API_KEY:
    print(f"✅ NVIDIA_API_KEY carregada: {NVIDIA_API_KEY[:15]}...")
else:
    print("⚠️ NVIDIA_API_KEY não encontrada.")

✅ NVIDIA_API_KEY carregada: nvapi-8tdO1NjXD...


## 2. Testar conexão com NVIDIA API

In [8]:

invoke_url = "https://integrate.api.nvidia.com/v1/chat/completions"

headers = {
    "Authorization": f"Bearer {NVIDIA_API_KEY}",
    "Content-Type": "application/json"
}

payload = {
    "model": "nvidia/nemotron-3-nano-30b-a3b",
    "messages": [
        {"role": "user", "content": "Explique o que é Deep Learning em 3 frases."}
    ],
    "temperature": 0.7,
    "max_tokens": 2000
}

response = requests.post(invoke_url, headers=headers, json=payload)

if response.status_code == 200:
    result = response.json()
    print("✅ Resposta do Nemotron:")
    print(result["choices"][0]["message"]["content"])
else:
    print(f"❌ Erro {response.status_code}: {response.text}")

✅ Resposta do Nemotron:

Deep Learning é uma área do aprendizado de máquina que utiliza redes neurais artificiais com múltiplas camadas ocultas para aprender representações hierárquicas de dados. Essas redes são capazes de capturar padrões complexos em grandes volumes de informações, como imagens, áudio e texto. O resultado é um sistema que pode realizar tarefas avançadas — como reconhecimento de objetos, tradução automática e geração de texto — com desempenho próximo ao humano.


## 3. Usando a biblioteca OpenAI (compatível)

In [10]:
# pip install openai
from openai import OpenAI

client = OpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key=NVIDIA_API_KEY
)

response = client.chat.completions.create(
    model="nvidia/nemotron-3-nano-30b-a3b",
    messages=[
        {"role": "user", "content": "Qual a diferença entre RNN e Transformer?"}
    ],
    temperature=0.7,
    max_tokens=300
)

print(response.choices[0].message.content)


## Diferenças fundamentais entre **RNN** (Redes Neurais Recorrentes) e **Transformer**

| Aspecto | RNN (Redes Neurais Recorrentes) | Transformer |
|---------|--------------------------------|-------------|
| **Processamento de sequência** | Processa tokens **um a um**, mantendo um estado interno (hidden state) que “repete” ao longo do tempo. Cada passo depende do passo anterior. | Processa a **entire sequence** de uma vez, usando **atenção** para relacionar todos os tokens entre si. Não há dependência temporal explícita nos cálculos. |
| **Dependência de longo alcance** | Difícil capturar dependências de longo prazo porque o gradiente pode desaparecer ou explodir ao longo de muitos passos. Estratégias como *truncamento*, *LSTM/GRU* ou *BPTT* ajudam, mas ainda há limitações. | Atenção permite **conectar diretamente** qualquer


## 4. Streaming (resposta em tempo real)

In [11]:
print("🤖 Nemotron: ", end="")

for chunk in client.chat.completions.create(
    model="nvidia/nemotron-3-nano-30b-a3b",
    messages=[{"role": "user", "content": "Liste 3 aplicações práticas de LLMs."}],
    temperature=0.7,
    max_tokens=200,
    stream=True
):
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end="", flush=True)
print()

🤖 Nemotron: 
**3 aplicações práticas de LLMs (Modelos de Linguagem de Grande Escala)**  

| # | Aplicação | Como funciona na prática | Benefícios principais |
|---|-----------|--------------------------|-----------------------|
| 1 | **Assistência e automação de atendimento ao cliente** | Chatbots e agentes virtuais alimentados por LLMs respondem a perguntas frequentes, resolvem tickets de suporte, criam scripts de vendas e personalizam interações em tempo real. | Redução de custos operacionais, disponibilidade 24 h, experiência de cliente mais rápida e consistente. |
| 2 | **Criação e tradução de conteúdo** | Redação automática de artigos, newsletters, posts para redes sociais, resumos de documentos e tradução multilíngue

IndexError: list index out of range

## 5. Chat com contexto (múltiplas mensagens)

In [12]:
messages = [
    {"role": "system", "content": "Você é um especialista em IA. Seja técnico e conciso."},
    {"role": "user", "content": "O que é atenção em transformers?"},
]

response = client.chat.completions.create(
    model="nvidia/nemotron-3-nano-30b-a3b",
    messages=messages,
    temperature=0.7
)

print(response.choices[0].message.content)


**Atenção em Transformers – visão técnica resumida**

1. **Objetivo**  
   - Permitir que cada token “olhe” para todos os outros tokens da sequência, aprendendo dependências de longo alcance sem recorrência.

2. **Cálculo básico (self‑attention)**  
   - Para cada posição \(i\) temos três vetores aprendidos:  
     \[
     \mathbf{Q}_i = \mathbf{X}_i \mathbf{W}_Q,\quad 
     \mathbf{K}_i = \mathbf{X}_i \mathbf{W}_K,\quad 
     \mathbf{V}_i = \mathbf{X}_i \mathbf{W}_V
     \]  
     onde \(\mathbf{X}_i\) é o embedding do token \(i\) e \(\mathbf{W}_Q,\mathbf{W}_K,\mathbf{W}_V\) são projeções lineares.  
   - **Score de atenção**:  
     \[
     \text{score}_{ij}= \frac{\mathbf{Q}_i \cdot \mathbf{K}_j}{\sqrt{d_k}}
     \]  
     (\(\sqrt{d_k}\) estabiliza o gradiente).  
   - **Peso de atenção** (softmax por linha):  
     \[
     \alpha_{ij}= \text{softmax}_j(\text{score}_{ij})
     \]  
   - **Saída da cabeça de atenção**:  
     \[
     \text{output}_i = \sum_j \alpha_{ij}\,\mathbf{V}

## 6. Parâmetros avançados

In [13]:
response = client.chat.completions.create(
    model="nvidia/nemotron-3-nano-30b-a3b",
    messages=[{"role": "user", "content": "Gere um título criativo para um artigo sobre IA."}],
    temperature=0.9,
    max_tokens=50,
    top_p=0.9
)

print(response.choices[0].message.content)

None


## Modelos Nemotron Disponíveis

| Modelo | Tamanho | Descrição |
|--------|---------|----------|
| `nvidia/nemotron-3-nano-30b-a3b` | 30B (3.5B ativos) | MoE híbrido, reasoning |
| `nvidia/nemotron-4-340b-instruct` | 340B | Modelo grande para tarefas complexas |

### Características:
- **Arquitetura**: Mamba2-Transformer Hybrid MoE
- **Idiomas**: Inglês, Alemão, Espanhol, Francês, Italiano, Japonês
- **Capacidades**: Reasoning, tool calling, chat, código